In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, RocCurveDisplay
from sklearn.metrics import make_scorer, roc_auc_score, recall_score, accuracy_score, confusion_matrix
from sklearn.inspection import permutation_importance

# Loading and cleaning Dataset

In [ ]:
database_file  =
output_location =

In [ ]:
def load_clean_dataset(loaded_df, na_perc_limit, columns_to_delete=[]):
    print(loaded_df.shape)
    loaded_df = loaded_df.drop(columns=columns_to_delete)
    tot = loaded_df.shape[0]
    print("Cleaning dataset... \n")
    for col in loaded_df.columns:
        na_per = 1-len(loaded_df[col].dropna())/tot
        if na_per > na_perc_limit:
            print(f"Column {col} --> %NaN = {na_per}. Removed")
            loaded_df = loaded_df.drop(columns=col)
    print("\n Dropping rows with NaNs...")
    loaded_df = loaded_df.dropna()
    print(f"\nFinal columns: {loaded_df.columns}")
    print(loaded_df.shape)
    return loaded_df

columns_to_delete = ["Unnamed: 0", "person_id", "fecha_ingreso_urgencias", "shock_septico", "foco", "sintoma_nan", "fecha_nacimiento", "codigo_postal"]

def combine_columns(df_to_clean, column_list, new_col_name):
    df_to_clean[new_col_name] = df_to_clean[column_list].sum(axis=1)
    clean_df = df_to_clean.drop(columns=column_list)
    return clean_df

df_to_clean = pd.read_csv(database_file)
hepatic_cols = [c for c in df_to_clean.columns if "hepatopatia" in c]
tumor_cols = [c for c in df_to_clean.columns if "cancer" in c]
for new_name, col_list in {"enf_hepaticas": hepatic_cols, "tumores": tumor_cols}.items():
    df_to_clean = combine_columns(df_to_clean, col_list, new_name)

processed_df = load_clean_dataset(
    loaded_df = df_to_clean,
    na_perc_limit = 0.1,
    columns_to_delete = columns_to_delete
)

### Splitting data into train/test

In [ ]:
X = processed_df.drop("sepsis", axis=1)
y = processed_df["sepsis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Defining models and hyperparameters for grid search

In [ ]:
log_dict = {
    "logistic_regression": {
        "model": LogisticRegression(),
        "params": {
            "solver": ["lbfgs", "liblinear", "saga"],
            "C": [0.1, 1, 10, 100, 250, 500],
            "penalty": ["l2", "l1", "elasticnet"],
            "max_iter": [100, 1000, 5000]
        }
    }
}
randfor_dict = {
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {
            "criterion": ["gini", "entropy", "log_loss"],
            "n_estimators": [10, 50, 100, 250, 500],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [5, 10, 20, 30, 50]
        }
    }
}
bayes_dict = {
    "bernoulli_bayes": {
        "model": BernoulliNB(),
        "params": {
            "alpha": [0.1, 0.5, 1.0, 1.5, 2.0],
            "fit_prior": [True, False],
            "binarize": [0.0, 0.5, 1.0, 1.5, 2.0]
        }
    }
}
svc_dict = {
    "svc": {
        "model": SVC(),
        "params": {
            "C": [0.1, 1, 10, 100],
            "kernel": ["linear", "rbf", "poly", "sigmoid"],
            "gamma": ["scale", "auto"]
        }
    }
}
grid_search_list = (log_dict, randfor_dict, bayes_dict, grad_boost_dict)
all_grid_models = {k:v for model_dict in model_list for k,v in model_dict.items()}

In [ ]:
from scipy.stats import uniform, loguniform, randint

log_dict = {
    "logistic_regression": {
        "model": LogisticRegression(),
        "params": {
            "solver": ["liblinear", "saga"],
            "C": loguniform(0.1, 500),
            "penalty": ["l2", "l1", "elasticnet"],
            "max_iter": [100, 1000, 5000]
        }
    }
}

randfor_dict = {
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {
            "criterion": ["gini", "entropy", "log_loss"],
            "n_estimators": randint(10, 501),
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": randint(5, 51)
        }
    }
}

bayes_dict = {
    "bernoulli_bayes": {
        "model": BernoulliNB(),
        "params": {
            "alpha": uniform(0.1, 2.0),
            "fit_prior": [True, False],
            "binarize": uniform(0.0, 2.0)
        }
    }
}


random_search_list = (log_dict, randfor_dict, bayes_dict, grad_boost_dict)
all_random_models = {k:v for model_dict in model_list for k,v in model_dict.items()}

### Defining search method arguments

f1 score is very common for clinical data as it indicates a balance between precision and recall

In [ ]:

random_search_args = {
    "search_model": RandomizedSearchCV,
    "search_params": {
        "cv":10, "n_iter":1000, "n_jobs":4, "scoring":{
            "f1": "f1",
            "AUC": "roc_auc",
            "Accuracy": "accuracy",
            "Recall": "recall"
        },
        "refit": "f1"
    }
}

grid_search_args = {
    "search_model": GridSearchCV,
    "search_params": {
        "cv":5, "scoring":{
            "f1": "f1",
            "AUC": "roc_auc",
            "Accuracy": "accuracy",
            "Recall": "recall"
        },
        "refit": "f1"
    }
}



### Training

In [ ]:
def cv_search(models_params, search_args):
    print("Given search args: ", search_args)
    scores_list = []
    search_engine = search_args["search_model"]
    search_params = search_args["search_params"]
    for model_name, mp in models_params.items():
        print(f"Starting cross-validation for {model_name}...")
        cv_classifier = search_engine(mp["model"], mp["params"], **search_params)
        cv_classifier.fit(X_train, y_train)
        print(f"Results for {model_name}:\n", cv_classifier.best_score_, cv_classifier.best_params_)
        scores_list.append({
            "model": model_name,
            "best_score": cv_classifier.best_score_,
            "best_params": cv_classifier.best_params_,
            "cv_results": cv_classifier.cv_results_
        })
    return scores_list

def get_feature_weights(best_model, X, y, n_repeats, state, score):
    return permutation_importance(best_model, X, y, n_repeats=n_repeats, random_state=state, scoring=score)

def run_testing(X, y, models_dict, search_args):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

    scores = cv_search(models_params=models_dict, search_args=search_args)

    print("Training finished")

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(scores, columns=["model", "best_score", "best_params", "cv_results"])
    results_df.to_csv(os.path.join(output_location, "results_df.csv"))

    # Select the best model and evaluate it on the test set
    best_model_name = results_df.loc[results_df["best_score"].idxmax()]["model"]
    best_model_params = results_df.loc[results_df["best_score"].idxmax()]["best_params"]

    best_model = models_dict[best_model_name]["model"]
    best_model.set_params(**best_model_params)
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]

    print(f"Found best model: {best_model_name} with parameters: {best_model_params}")
    print("Scores table: \n", classification_report(y_test, y_pred))
    try:
        feature_weights = get_feature_weights(best_model, X_train, y_train, n_repeats=10, state=99, score="f1")
    except Exception:
        feature_weights = {}
    if feature_weights:
        importance_df = pd.DataFrame({y:x for x,y in sorted(zip(feature_weights["importances_mean"], X_train.columns), reverse=True)}, index=[0])
        importance_df.to_csv(os.path.join(output_location, f"{best_model_name}_importance_df.csv"))
    else:
        print(f"Could not extract importance values for best model {best_model_name}:{best_model_params}")
    class_report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
    class_report_df["roc_auc"] = roc_auc_score(y_test, y_pred_prob)
    class_report_df.to_csv(os.path.join(output_location, f"{best_model_name}_class_report.csv"))
    return results_df, class_report_df

models_dict = all_random_models

search_args = random_search_args
X = processed_df.drop("sepsis", axis=1)
y = processed_df["sepsis"]
results_df, class_report_df = run_testing(X, y, models_dict, search_args)


# Plot scores distribution along cross-validation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_cv_results(cv_results_df):
    fig, ax = plt.subplots(figsize=(12,6))
    for col in ["mean_test_AUC", "mean_test_f1", "mean_test_Accuracy", "mean_test_Recall"]:
        sns.kdeplot(data=cv_results_df, x=col, label=col)
    ax.legend()
    
    plt.tight_layout()
    plt.show()

for idx, row in results_df.iterrows():
    cv_results = pd.DataFrame(row["cv_results"])
    plot_cv_results(cv_results)


# Gradient Boost and other approaches

In [ ]:
from sklearn.linear_model import SGDCLassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
import xgboost as xgb

In [ ]:
grad_boost_dict = {
    "gradient_boosting": {
        "model": GradientBoostingClassifier(),
        "params": {
            "n_estimators": randint(100, 500),
            "learning_rate": loguniform(0.001, 0.1),
            "max_depth": randint(3, 10),
            "subsample": uniform(0.7, 1),
            "min_samples_split": randint(5, 20)
        }
    }
}

sgdc_dict = {
    "sgdc_classifier": {
        "model": SGDCLassifier(),
        "params": {
            "loss": ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron"],
            "alpha": loguniform(0.0001, 0.1),
            "max_iter": randint(1000, 20000),
            "early_stopping": [True]
        }
    }
}